In [1]:
import pandas as pd
import numpy as np
import re

import io
import csv
import sys 

# Progress bar
from tqdm.auto import tqdm

maxInt = sys.maxsize
while True:
    # decrease the maxInt value by factor 10 as long as the OverflowError occurs.
    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

from implementations_NB01 import*

## Data reading

In [2]:
header_traj_0800_0830, all_lines_0800_0830 = read_traj_csv('data/monday_d8_0800_0830_OD_trajectories.csv')
header_traj_0830_0900, all_lines_0830_0900 = read_traj_csv('data/monday_d8_0830_0900_OD_trajectories.csv')
header_traj_0900_0930, all_lines_0900_0930 = read_traj_csv('data/monday_d8_0900_0930_OD_trajectories.csv')
header_traj_0930_1000, all_lines_0930_1000 = read_traj_csv('data/monday_d8_0930_1000_OD_trajectories.csv')
header_traj_1000_1030, all_lines_1000_1030 = read_traj_csv('data/monday_d8_1000_1030_OD_trajectories.csv')
all_lines1 = all_lines_0800_0830 + all_lines_0830_0900 + all_lines_0900_0930 + all_lines_0930_1000 + all_lines_1000_1030

header_traj_0800_0830, all_lines_0800_0830 = read_traj_csv('data/tuesday_d8_0800_0830_OD_trajectories.csv')
header_traj_0830_0900, all_lines_0830_0900 = read_traj_csv('data/tuesday_d8_0830_0900_OD_trajectories.csv')
header_traj_0900_0930, all_lines_0900_0930 = read_traj_csv('data/tuesday_d8_0900_0930_OD_trajectories.csv')
header_traj_0930_1000, all_lines_0930_1000 = read_traj_csv('data/tuesday_d8_0930_1000_OD_trajectories.csv')
header_traj_1000_1030, all_lines_1000_1030 = read_traj_csv('data/tuesday_d8_1000_1030_OD_trajectories.csv')
all_lines2 = all_lines_0800_0830 + all_lines_0830_0900 + all_lines_0900_0930 + all_lines_0930_1000 + all_lines_1000_1030

header_traj_0830_0900, all_lines_0830_0900 = read_traj_csv('data/wednesday_d8_0830_0900_OD_trajectories.csv')
header_traj_0900_0930_01, all_lines_0900_0930_01 = read_traj_csv('data/wednesday_d8_0900_0930_OD_trajectories_01.csv')
header_traj_0900_0930_02, all_lines_0900_0930_02 = read_traj_csv('data/wednesday_d8_0900_0930_OD_trajectories_02.csv')
header_traj_0930_1000_01, all_lines_0930_1000_01 = read_traj_csv('data/wednesday_d8_0930_1000_OD_trajectories_01.csv')
header_traj_0930_1000_02, all_lines_0930_1000_02 = read_traj_csv('data/wednesday_d8_0930_1000_OD_trajectories_02.csv')
header_traj_1000_1030_01, all_lines_1000_1030_01 = read_traj_csv('data/wednesday_d8_1000_1030_OD_trajectories_01.csv')
header_traj_1000_1030_02, all_lines_1000_1030_02 = read_traj_csv('data/wednesday_d8_1000_1030_OD_trajectories_02.csv')
header_traj_1030_1100, all_lines_1030_1100 = read_traj_csv('data/wednesday_d8_1030_1100_OD_trajectories.csv')
all_lines3 = all_lines_0830_0900 + all_lines_0900_0930_01+all_lines_0900_0930_02 + all_lines_0930_1000_01+all_lines_0930_1000_02 + all_lines_1000_1030_01+all_lines_1000_1030_02+all_lines_1030_1100

header_traj_0800_0830, all_lines_0800_0830 = read_traj_csv('data/thursday_d8_0800_0830_OD_trajectories.csv')
header_traj_0830_0900, all_lines_0830_0900 = read_traj_csv('data/thursday_d8_0830_0900_OD_trajectories.csv')
header_traj_0900_0930, all_lines_0900_0930 = read_traj_csv('data/thursday_d8_0900_0930_OD_trajectories.csv')
header_traj_0930_1000, all_lines_0930_1000 = read_traj_csv('data/thursday_d8_0930_1000_OD_trajectories.csv')
header_traj_1000_1030_01, all_lines_1000_1030_01 = read_traj_csv('data/thursday_d8_1000_1030_OD_trajectories_01.csv')
header_traj_1000_1030_02, all_lines_1000_1030_02 = read_traj_csv('data/thursday_d8_1000_1030_OD_trajectories_02.csv')
all_lines4 = all_lines_0800_0830 + all_lines_0830_0900 + all_lines_0900_0930 + all_lines_0930_1000 + all_lines_1000_1030_01+all_lines_1000_1030_02
all_lines = all_lines1+all_lines2+all_lines3+all_lines4

In [3]:
update_track_id(all_lines) #changes the track ID of each vehicle with an unique int

In [4]:
len(all_lines)

9429

## Separating the data

**Inputs:**
- Track ID
- Class of vehicle
- Destination
- Origin
- Total distance parcoured by the vehicle
- Average speed
- Average tangential accerelation
- Average lateral acceleration
- Number of lane changes

For the class of vehicle, the following enconding was used: 
1 if a car, 2 if a taxi, 3 if a motorcyle, 4 if a bus, 5 if a medium vehicle, 6 if a heavy vehicle

**Outputs:** 
- Track ID
- Distance parcoured in the last lane

In [5]:
# Creation of a dataFrame with headers

dict = {'Track_ID':[-1],
        'Class_vehicle':['car'],
        'Destination_D':['Destination 2'],
        'Origin_lane':[' Lane 2'],
        'Total_distance':[46],
        'Avg_speed':[0],
        'Avg_Acc_tan':[0],
        'Avg_Acc_lat':[0],
        'Nbr_LaneChange':[0],
        'Distance_last_lane':[0]
       }
df = pd.DataFrame(dict)

In [6]:
#Append the infos of all vehicle, needed for the imputs and the outputs of the NN
# The structure is explained before
#be patient, can take between 5-10 min
for veh in tqdm(range(len(all_lines))):
    infos=list()
    dist = compute_dist_last_lane_ID_i(all_lines,veh)
    info_init_veh ,info_measured_veh = infos_vehicule_i(all_lines,veh)
    infos.append(info_init_veh[0]) #Track ID
    infos.append(info_init_veh[1]) #Class Vehicule
    infos.append(info_init_veh[4]) #Destination
    infos.append(info_measured_veh[0][-1]) #Origin Lane
    infos.append(info_init_veh[6]) #total distance
    infos.append(info_init_veh[7]) #Average speed
    infos.append(np.mean(info_measured_veh[:,3])) #Avg Acc_tan
    infos.append(np.mean(info_measured_veh[:,4])) #Avg Acc_lat
    infos.append(compute_chgmnt_ID_i(all_lines,veh)) #Nbr_LaneChange
    infos.append(dist) #Distance
    df.loc[len(df.index)] = infos 
df = df.drop(0)

  0%|          | 0/9429 [00:00<?, ?it/s]

In [7]:
#Visualisation
df.head(5)
#df.shape = (9429, 10)

,Track_ID,Class_vehicle,Destination_D,Origin_lane,Total_distance,Avg_speed,Avg_Acc_tan,Avg_Acc_lat,Nbr_LaneChange,Distance_last_lane
1,0,Car,Destination 3,"""Lane 2""",345.18,25.093983,-0.155225,-0.008384,1,0.000000
2,1,Car,Destination 3,"""Lane 1""",344.73,30.447420,-0.220076,-0.010061,1,0.000000
3,2,Car,Destination 3,"""Lane 1""",344.85,32.877334,-0.200211,-0.018431,1,0.000000
4,3,Car,Destination 2,"""Lane 2""",333.33,27.198438,-0.117522,0.103160,2,39.407715
5,4,Car,Destination 3,"""Lane 1""",344.91,28.636222,-0.175632,-0.004028,0,0.000000


In [8]:
# In order to have a number assigned to the type of vehicle
conditions_class = [
    (df['Class_vehicle']==' Car'),
    (df['Class_vehicle']==' Taxi'),
    (df['Class_vehicle']==' Motorcycle'),
    (df['Class_vehicle']==' Bus'),
    (df['Class_vehicle']==' Medium Vehicle'),
    (df['Class_vehicle']==' Heavy Vehicle')
    ]
values_class = [1,2,3,4,5,6]

df['Class'] = np.select(conditions_class, values_class)

In [9]:
# In order to have a number assigned to the destination
conditions_destination = [
    (df['Destination_D']==' Destination 1'),
    (df['Destination_D']==' Destination 2'),
    (df['Destination_D']==' Destination 3')
    ]
values_destination = [1,2,3]

df['Destination'] = np.select(conditions_destination, values_destination)

In [10]:
# In order to have a number assigned to the origine's lane
conditions_origin = [
    (df['Origin_lane']==' "Lane 1"'),
    (df['Origin_lane']==' "Lane 2"'),
    (df['Origin_lane']==' "Lane 3"')
    ]
values_origin = [1,2,3]

df['Origin'] = np.select(conditions_origin, values_origin)

In [11]:
df.head(5)

,Track_ID,Class_vehicle,Destination_D,Origin_lane,Total_distance,Avg_speed,Avg_Acc_tan,Avg_Acc_lat,Nbr_LaneChange,Distance_last_lane,Class,Destination,Origin
1,0,Car,Destination 3,"""Lane 2""",345.18,25.093983,-0.155225,-0.008384,1,0.000000,1,3,2
2,1,Car,Destination 3,"""Lane 1""",344.73,30.447420,-0.220076,-0.010061,1,0.000000,1,3,1
3,2,Car,Destination 3,"""Lane 1""",344.85,32.877334,-0.200211,-0.018431,1,0.000000,1,3,1
4,3,Car,Destination 2,"""Lane 2""",333.33,27.198438,-0.117522,0.103160,2,39.407715,1,2,2
5,4,Car,Destination 3,"""Lane 1""",344.91,28.636222,-0.175632,-0.004028,0,0.000000,1,3,1


In [12]:
df = df.drop("Class_vehicle",axis=1)
df = df.drop("Destination_D",axis=1)
df = df.drop("Origin_lane",axis=1)

In [13]:
df.head(5)

,Track_ID,Total_distance,Avg_speed,Avg_Acc_tan,Avg_Acc_lat,Nbr_LaneChange,Distance_last_lane,Class,Destination,Origin
1,0,345.18,25.093983,-0.155225,-0.008384,1,0.000000,1,3,2
2,1,344.73,30.447420,-0.220076,-0.010061,1,0.000000,1,3,1
3,2,344.85,32.877334,-0.200211,-0.018431,1,0.000000,1,3,1
4,3,333.33,27.198438,-0.117522,0.103160,2,39.407715,1,2,2
5,4,344.91,28.636222,-0.175632,-0.004028,0,0.000000,1,3,1


In [14]:
#df.to_csv("input_files/Initial_input.csv")